<a href="https://colab.research.google.com/github/LuanPCunha/TCC/blob/main/RotinaDeTreinamentoModelosDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install tensorflow
!pip   uninstall tensorflow -y

Found existing installation: tensorflow 2.8.0
Uninstalling tensorflow-2.8.0:
  Successfully uninstalled tensorflow-2.8.0


In [ ]:
! pip install keras==2.7.0

In [ ]:
!pip show keras

Name: keras
Version: 2.8.0
Summary: Deep learning for humans.
Home-page: https://keras.io/
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache 2.0
Location: /usr/local/lib/python3.7/dist-packages
Requires: 
Required-by: tensorflow, keras-vis


In [2]:
import pandas as pd
import numpy as np
import string, re
import itertools
import nltk
import plotly.offline as py
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.callbacks import EarlyStopping
py.init_notebook_mode(connected=True)
%matplotlib inline
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

In [ ]:
# import tensorflow as tf
# from tensorflow import keras, SparseTensor
# import pandas as pd
# import numpy as np
# import string, re
# import itertools
# import nltk
# import plotly.offline as py
# import plotly.graph_objs as go
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.naive_bayes import MultinomialNB
# from sklearn.metrics import accuracy_score
# #from keras.preprocessing.text import Tokenizer
# #from keras.preprocessing.sequence import pad_sequences
# #from keras.models import Sequential
# #from keras.layers import Dense, Embedding, LSTM
# #from keras.callbacks import EarlyStopping
# py.init_notebook_mode(connected=True)
# %matplotlib inline
# from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn import decomposition, ensemble
# import xgboost, textblob
# #from keras.preprocessing import text, sequence
# #from keras import layers, models, optimizers

In [3]:
# CAMINHO DOS DADOS

#Base1
PATH_BASE1 = "/content/drive/MyDrive/TCC/dados/processadas/Base1_classificada.csv"
#Base2
PATH_BASE2 = "/content/drive/MyDrive/TCC/dados/processadas/Base2_classificada.csv"

# Lendo Base não balanceada

In [40]:
trainDF = pd.read_csv(PATH_BASE1, index_col=0)  #trainDF = trainDF.sample(n=1000) #PEGANDO 1000 REGISTROS DA BASE PARA TESTE, REMOVER ISSO DEPOIS

# Dropa colunas que não serão utilizadas
trainDF.drop(['id_str', 'created_at','text','entities.hashtags','place.full_name','classificacao_badword'], axis='columns', inplace=True)

# Renomeia as colunas que sobraram
trainDF.rename(columns = {'text_limpo': 'text', 'classificacao_hate': 'label'}, inplace = True)
trainDF.head(5)

,text,label
1,retwet eu queria compartilhar mas antes eu gos...,not_hate
2,gostei da pulseira e da educação dele nossaaa ...,not_hate
3,esses não irão ao encontro e beijing,not_hate
5,e eu me questionando quem é esse blogueiro sem...,not_hate
6,renan caloteiro desonesto vejam link abaixo,not_hate


In [5]:
# Pega a média de caracteres dos tweets de toda a base
average_len = trainDF['text'].apply(lambda x: len(x)).mean()
average_len

80.85359867330017

In [6]:
# Agrupa dados por label
trainDF.groupby('label').count()

,text
label,
hate,5177
not_hate,55123


In [27]:
# convert output  hate =1 not_hate =0
trainDF.label = [1 if each == "hate" else 0 for each in trainDF.label] 

In [ ]:
categorias_desejadas = [0, 1] # !!!
df_filtrado = trainDF[trainDF.label.isin(categorias_desejadas)]
df_filtrado.shape

(60300, 2)

In [ ]:
df_filtrado.head(5)

,text,label
1,retwet eu queria compartilhar mas antes eu gos...,0
2,gostei da pulseira e da educação dele nossaaa ...,0
3,esses não irão ao encontro e beijing,0
5,e eu me questionando quem é esse blogueiro sem...,0
6,renan caloteiro desonesto vejam link abaixo,0


In [ ]:
def balanceia_dados(df, verbose=0):
    # Esta funcao recebe um dataframe original e retorna um dataset
    # contendo um numero igual de noticias das categorias
    # por simplicidade, vou supor que são somente duas categorias.
    global categorias_desejadas
    df_balanceado = df.copy()
    
    # cria um dataframe para cada classe. Assim podemos remover o excesso
    df_F = df_balanceado.loc[df_balanceado.label == categorias_desejadas[0]]
    df_T = df_balanceado.loc[df_balanceado.label == categorias_desejadas[1]]

    num_amostras_F, num_amostras_T = df_F.shape[0], df_T.shape[0]
    num_amostras_a_remover = abs(num_amostras_F - num_amostras_T)
    
    # remove amostras da classe majoritaria
    if num_amostras_F > num_amostras_T:
        #print(df_F[:num_amostras_a_remover)
        df_F = df_F[num_amostras_a_remover:]
        
    elif num_amostras_T > num_amostras_F:
        #
        df_T = df_T[num_amostras_a_remover:]

    return pd.concat([df_F, df_T])

In [ ]:
# Aplica função de balancear os dados
dataBC = balanceia_dados(df_filtrado)
dataBC.shape

(10354, 2)

In [ ]:
dataBC.head()

,text,label
74562,só esse parque shoping em sulacap mesmo embora...,0
74566,jungkok você quer matar o fandom inteiro do co...,0
74569,amiga eu ja tinha desistido ha anos com a conv...,0
74570,sim gente é só sujeira no celular o seu ídolo ...,0
74573,já tenho 2 chá de bebê para ir k,0


In [ ]:
dataBC.groupby('label').count()

In [ ]:
# DATA BALANCEADO
trainDF = dataBC
trainDF.reset_index(drop=True, inplace=True) # reseta index
trainDF.shape

(10354, 2)

In [ ]:
trainDF.head(5)

,text,label
1,retwet eu queria compartilhar mas antes eu gos...,not_hate
2,gostei da pulseira e da educação dele nossaaa ...,not_hate
3,esses não irão ao encontro e beijing,not_hate
5,e eu me questionando quem é esse blogueiro sem...,not_hate
6,renan caloteiro desonesto vejam link abaixo,not_hate


# Doideira que Thalessa fez seguindo o cara do vídeo

In [42]:
#Usando DF não balanceado
trainDF.groupby('label').count()

,text
label,
hate,5177
not_hate,55123


In [43]:
trainDF['label'].head()

1    not_hate
2    not_hate
3    not_hate
5    not_hate
6    not_hate
Name: label, dtype: object

In [44]:
# Salva os registros da coluna 'text' do dataframe em uma lista
text = []
trainDF['text'].apply(lambda x: text.append(x))

1        None
2        None
3        None
5        None
6        None
         ... 
82244    None
82245    None
82246    None
82247    None
82248    None
Name: text, Length: 60300, dtype: object

In [31]:
# converte average_len em inteiro
average_len = int(average_len)
print(average_len)

80


In [45]:
# Para que transformar os tuítes em matriz de números
tokenizer = Tokenizer(num_words=5000, split=' ')

tokenizer.fit_on_texts(text)
X = tokenizer.texts_to_sequences(text)  

# limitando o tamanho das sequências com base no tamanho máximo que especificamos na variável "max_sequence_length"
X = pad_sequences(X, maxlen=average_len)

In [34]:
# TEM QUE VER ISSO AQUI, NÃO FUNCIONA POW PANDAS
# Transforma os label em 0 e 1
Y = pd.get_dummies(trainDF['label']).values

In [46]:
# convert output  hate =1 not_hate =0
trainDF.label = [1 if each == "hate" else 0 for each in trainDF.label] 

In [47]:
Y = trainDF['label']

In [49]:
print(Y)

(60300,)


In [33]:
print(X)

[[   0    0    0 ...   87   23  215]
 [   0    0    0 ...  223   53 2807]
 [   0    0    0 ...   62  985    5]
 ...
 [   0    0    0 ...  113   78  332]
 [   0    0    0 ...   24   18 3729]
 [   0    0    0 ...    9   96  956]]


In [50]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(X, Y, test_size=0.25, random_state=2)
#X_train, X_test, Y_train, Y_test

In [51]:
train_x.shape

(45225, 80)

In [54]:
# SHALLOW NET

def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)

    classifier.compile(loss='binary_crossentropy', optimizer='sgd', metrics = ['accuracy'])
    
    return classifier 

# Modelo criado
model = create_model_architecture(average_len)

# Modelo sendo treinado/executado
epochs = 5
batch_size = 32
hist = model.fit(
      train_x, 
      train_y, 
      validation_data=(valid_x, valid_y),
      epochs=epochs,
      batch_size=batch_size, 
      shuffle=True,
      verbose=1)

Epoch 1/5
1414/1414 [==============================] - 5s 3ms/step - loss: 10.7743 - accuracy: 0.8454 - val_loss: 6.2832 - val_accuracy: 0.8948
Epoch 2/5
1414/1414 [==============================] - 4s 3ms/step - loss: 3.8643 - accuracy: 0.8534 - val_loss: 3.2159 - val_accuracy: 0.8434
Epoch 3/5
1414/1414 [==============================] - 4s 3ms/step - loss: 1.9995 - accuracy: 0.8609 - val_loss: 2.7782 - val_accuracy: 0.6382
Epoch 4/5
1414/1414 [==============================] - 4s 3ms/step - loss: 1.1152 - accuracy: 0.8693 - val_loss: 0.9666 - val_accuracy: 0.8637
Epoch 5/5
1414/1414 [==============================] - 4s 3ms/step - loss: 0.7277 - accuracy: 0.8774 - val_loss: 0.5916 - val_accuracy: 0.8902


In [55]:
scores = model.evaluate(valid_x, valid_y, verbose = 0, batch_size = batch_size)
print("Acc: %.2f%%" % (scores[1]*100))

Acc: 89.02%



# Divisão Trainamento/Teste






---



In [ ]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'], test_size=0.25, random_state=2)

# label encode the target variable 
encoder = preprocessing.LabelEncoder() # !!!
train_y_enc = encoder.fit_transform(train_y)
valid_y_enc = encoder.fit_transform(valid_y)
print(valid_y_enc)

[1 0 1 ... 1 0 0]


In [ ]:
train_y.shape

(45225, 2)

# Baseline

## NB

In [ ]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(trainDF['text'])

# transform the training and validation data using count vectorizer object
xtrain_count =  count_vect.transform(train_x)
xvalid_count =  count_vect.transform(valid_x)

In [ ]:
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=4000)
tfidf_vect.fit(trainDF['text'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=4000)
tfidf_vect_ngram.fit(trainDF['text'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=4000)
tfidf_vect_ngram_chars.fit(trainDF['text'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

In [ ]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_count, train_y_enc, xvalid_count)
print ("NB, Count Vectors: ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y_enc, xvalid_tfidf)
print ("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y_enc, xvalid_tfidf_ngram)
print ("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y_enc, xvalid_tfidf_ngram_chars)
print ("NB, CharLevel Vectors: ", accuracy)

## Shallow net (tbm baseline)

In [ ]:
def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    # opt = optimizers.Adam(learning_rate=0.01)
    opt = optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False, name="SGD")

    classifier.compile(optimizer=opt, loss='binary_crossentropy', metrics = ['acc'])
    
    return classifier 



In [ ]:
xtrain_tfidf.shape

(7765, 4000)

In [ ]:
classifier = create_model_architecture(xtrain_tfidf.shape[1])
accuracyNN = train_model(classifier, xtrain_tfidf, train_y, xvalid_tfidf, is_neural_net=True)
print ("NN, Ngram Level TF IDF Vectors",  accuracyNN)

AttributeError: ignored

# word embeddings

In [ ]:
from gensim.models import KeyedVectors

In [ ]:
model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/PUBLICAÇÕES/PUBLICAÇÕES 2020/CISTI-2020/cbow_s50_filtrado.txt')

In [ ]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('/content/drive/My Drive/PUBLICAÇÕES/PUBLICAÇÕES 2020/CISTI-2020/cbow_s50_filtrado.txt')):
    values = line.split()
    embeddings_index[values[0]] = numpy.asarray(values[1:], dtype='float32')

In [ ]:
# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(trainDF['text'])
word_index = token.word_index


In [ ]:
tam= len(word_index)
tam

In [ ]:
# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=100)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=100)

In [ ]:
# create token-embedding mapping
embedding_matrix = numpy.zeros((len(word_index) + 1, 50))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# CNN model

In [ ]:
def create_cnn():
    # Add an Input Layer
    input_layer = keras.layers.Input((100, ))

    # Add the word embedding Layer
    embedding_layer = keras.layers.Embedding(len(word_index) + 1, 50, weights=[embedding_matrix], trainable=True)(input_layer)
    embedding_layer = keras.layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = keras.layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = keras.layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = keras.layers.Dense(64, activation="relu")(pooling_layer)
    output_layer1 = keras.layers.Dropout(0.25)(output_layer1)
    output_layer2 = keras.layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model = keras.models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=sgd, loss='mean_squared_error', metrics = ['acc', 'mae'])
    return model



In [ ]:
classifier = create_cnn()
accuracyCNN = train_model(classifier,train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("CNN, Word Embeddings - accuracy",  accuracyCNN)

# LSTM

In [ ]:
def create_rnn_lstm():
# Add an Input Layer
    input_layer = layers.Input((100, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 50, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', metrics=['acc', 'mae'])
    return model



In [ ]:
classifier = create_rnn_lstm()
accuracyLSTM = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("RNN-LSTM, Word Embeddings",  accuracyLSTM)

# RNN gru

In [ ]:
def create_rnn_gru():
  # Add an Input Layer
    input_layer = layers.Input((100, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 50, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the GRU Layer
    lstm_layer = layers.GRU(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(64, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    #model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    #model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', metrics = ['acc'])
    model.compile(optimizer=sgd, loss='mean_squared_error', metrics = ['acc','mae'])
    
    return model

In [ ]:
classifier = create_rnn_gru()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("RNN-GRU, Word Embeddings",  accuracy)





## Bidirectional RNN

In [ ]:
def create_bidirectional_rnn():
   # Add an Input Layer
    input_layer = layers.Input((150, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 50, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.GRU(100))(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(64, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', metrics=['acc','mae'])
    
    return model

In [ ]:
classifier = create_bidirectional_rnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("RNN-Bidirectional, Word Embeddings",  accuracy)

##  - CRNN Convolutional recurrent neural network

In [ ]:
def create_rcnn():
  # Add an Input Layer
    input_layer = layers.Input((70, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 50, weights=[embedding_matrix], trainable=True)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    # Add the recurrent layer
    rnn_layer = layers.Bidirectional(layers.GRU(50, return_sequences=True))(embedding_layer)
    
    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', metrics=['acc'])
    
    return model

In [ ]:
classifier = create_rcnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("CNN, Word Embeddings",  accuracy)